In [1]:
import codecs
import glob
import os
import pandas as pd
import pycountry
from io import StringIO

import seaborn as sns

# find target path for data files, assuming the notebook is in the right place
data_path = _dh[0]
assert data_path.endswith(os.path.join('em-2020','data'))

output_path = os.path.join(data_path, 'ChinnIto_reshaped.csv')

In [2]:
kaopen_raw = pd.read_excel(os.path.join(data_path, 'kaopen_2018.xls'))
kaopen_raw.head()

,cn,ccode,country_name,year,kaopen,ka_open
0,111,USA,United States,1970,2.333585,1.0
1,111,USA,United States,1971,2.333585,1.0
2,111,USA,United States,1972,2.333585,1.0
3,111,USA,United States,1973,2.333585,1.0
4,111,USA,United States,1974,2.333585,1.0


In [3]:
kaopen_countries = kaopen_raw[['ccode', 'country_name']].drop_duplicates()
mapped_countries = kaopen_countries.ccode.map(lambda x: pycountry.countries.get(alpha_3=x))
kaopen_countries.loc[mapped_countries.isnull()]

,ccode,country_name
2646,ANT,Netherlands Antilles
5145,ZAR,"Congo, Dem. Rep."
8869,ROM,Romania


In [4]:
display(pycountry.countries.get(alpha_3='ROU'))
display(pycountry.countries.get(alpha_3='COD'))

Country(alpha_2='RO', alpha_3='ROU', name='Romania', numeric='642')

Country(alpha_2='CD', alpha_3='COD', name='Congo, The Democratic Republic of the', numeric='180')

In [5]:
kaopen_raw.ccode.replace('ZAR', 'COD', inplace=True)
kaopen_raw.ccode.replace('ROM', 'ROU', inplace=True)

kaopen_raw = kaopen_raw.loc[kaopen_raw.ccode != 'ANT']

assert not kaopen_raw[['ccode', 'year']].duplicated().any()

In [6]:
kaopen_reshaped = kaopen_raw[['ccode', 'year', 'kaopen']]
kaopen_reshaped.columns = ['Country Code', 'Year', 'KAOPEN']
kaopen_reshaped.set_index(['Country Code', 'Year'], inplace=True)
kaopen_reshaped.head()

KAOPEN
Country Code Year          
USA          1970  2.333585
             1971  2.333585
             1972  2.333585
             1973  2.333585
             1974  2.333585

In [7]:
kaopen_reshaped.to_csv(output_path)